# Databases

![](https://github.com/cs109/2015/raw/master/Lectures/Lecture4/ds.png)

>It took about three years before the BellKor’s Pragmatic Chaos team managed to win the prize ... The winning algorithm was ... so complex that it was never implemented by Netflix.

### DATA ENGINEERING

- compute: code, python, R, julia, spark, hadoop
- storage/database: git, SQL, NoSQL, HBase, disk, memory
- devops: AWS, docker, mesos, repeatability
- product: database, web, API, viz, UI, story

Different at different scales....

### Data Pipelines these days

If you are a datascientist or enginner in any small to mid sized company with a web presence, you might see something like this in your data infrastructure.

This example is from @wrobstory: the SIMPLE pipelines:

![](https://dl.dropboxusercontent.com/u/75194/simplepipe.png)

If you are a data engineer, you'll be expected to architect the systems which take external data, and internal data, and combine them into databases which are both the source and target for analysis.

The critical component of all this architecture is the database.

### Why learn databases

- you shouldnt really implement one: very hard to get right
- but you must understand how they work
- data storage/munging are not just database concerns, Eg: Apache Parquet, array, dplyr, pandas
- choosing a storage engine that is ok for your program
- understanding query performance
- transaction processing is not optimal for analytics

### What kind of data access do you need?

- relational: pandas, SQL: Postgres, sqlite, Hbase, VoltDB
- document oriented: MongoDB, CouchDB
- key-value: Riak, Memcached, leveldb
- graph oriented: Neo4J
- in-memory data-structure server: Redis

### Relational vs Document/key-value

- today both are highly used: we have *polyglot persistence*
- Mongo/Couch/etc are document oriented, store JSON documents
- these have a higher locality of data: its like a really wide row with hierarchy
- normalization vs denormalization

### Relational Model

- a relation (table) is a collection of tuples. Each tuple is called a *row*
- A collection of tables related to each other through common data values.
- Everything in a column is values of one attributes
- A cell is expected to be atomic
- Tables are related to each other if they have columns called keys which represent the same values
- SQL a declarative model: a query optimizer decides how to execute the query (if a field range covers 80% of values, should we use the index or the table?). Also parallelizable
- *shredding* splits a document into multiple tables due to normalization

![](https://github.com/cs109/2015/raw/master/Lectures/Lecture4/contributors.png)

![](https://github.com/cs109/2015/raw/master/Lectures/Lecture4/candidates.png)

### Key-Value Model

- like a dictionary
- the database is the index

### Document Model

- stores nested records
- bad for many-to-many
- storage locality good for access, bad for writing
- couch, mongo, etc

### Normalization vs denormalization

- normalization is the use of pointers (id's from other tables) to point elsewhere (make tables)
- denormalization is the following and expansion of the pointer
- how would you represent this for the contributes and candidates above? (make JSON)

## Components to a database

- client connection manager: what to do with incomings
- transactional storage
    - storage data structures and the log
    - transactions and ACID: atomicity, consistency, isolation, durability
- process model: coroutines, threads, processes
- query model and language: query optimization

![](https://dl.dropboxusercontent.com/u/75194/dbmscomponents.png) (DBMS components from Hellerstein at al: Architecture of a Database System: circa 2007)

![](https://github.com/cs109/2015/raw/master/Lectures/Lecture4/sqlexecution.png)

(from 7 databases in 7 weeks)

### Indexing and Databases

- an additional structure derived from the primary data
- however a clustered index may actually store the data
- this happens often in key-value databases, but can happen in tree based indexes as well
- there is overhead on writes: indexes speed up queries but slow down writes

### Binary Search Trees

We've seen sorting so far. 

Whever we want to maintain our search dataset in memory, sorted, we use something like a Binary Search Tree instead. 

They perform well with dynamic data where insertions and deletions are frequent, because of the so called $O(height)$ guarantees. 

### Search Trees Supported operations

Whenever we encounter a new data structure we should write down their ops and the performance of these ops. 

The idea for BSTs is to be like a sorted array but provide fast(logarithmic) inserts and deletes.

- **Search** O(h)
- **Select**(Order Statistic) O(h) : Needs Augmented Tree, up from O(1) on sorted arrays
- **Minimum/Maximum** O(h), up from O(1) on sorted arrays
- **Predecessor/Successor** O(h), up from O(1) on sorted arrays
- **Rank** or Index O(h): Needs Augmented Tree
- **In-Order Traversal** O(n)
- **Insert** O(h)
- **Delete** O(h)

If we guarantee a balanced Tree:

$$ h = lg(n)$$

and thus most of these ops are $O(lg(n))$.

And we dont have to pay the piper on the sort...

![](bstproperty.png)

![](oninserts.png)

### Btrees

![](https://dl.dropboxusercontent.com/u/75194/btree1q.png)

(from https://loveforprogramming.quora.com/Memory-locality-the-magic-of-B-Trees)

- "A linked sorted distributed range array with predefined sub array size which allows searches, sequential access, insertions, and deletions in logarithmic time. "
- it is a generalization of a binary tree
- but the branching factor is much higher, and the depth thus smaller
- btrees break database into pages, and read-or-write one page at a time. A page is about 4k in size (see https://www.tutorialspoint.com/operating_system/os_virtual_memory.htm )
- leaf pages contain all the values and may represent a clustered index
- the pointers in a btree are disk based pointers

![](https://dl.dropboxusercontent.com/u/75194/btree1.png)
(from designing data intensive applications)

When we update a key, a split can happen

![](https://dl.dropboxusercontent.com/u/75194/btree2.png)

(from designing data intensive applications)

This is an in-place modification. The data structure is mutable. This can cause issues for transactions, and must be dealt with. 

Both splits and writing in-place are dangerous, so its normal for b-tree implementations to have a WAL, or write ahead log (such a log can also be used to manage transactions). Every operation on the btree is appended to this log file.

![](https://dl.dropboxusercontent.com/u/75194/btree2q.png)

In B+ trees, pointers amonst the leaf nodes make for an easier linear scan.

![](https://dl.dropboxusercontent.com/u/75194/bplus.png)

(from https://loveforprogramming.quora.com/Memory-locality-the-magic-of-B-Trees)

### Handling the different workloads

- for smaller sizes any relational db will do
- currently vendors focus on one or the other, not both
- MS and SAP HANA support both but with different storage engines
- OLTP need to be highly available, low latency
- SQL (or any Pandasish syntax) is good for drilling down
- warehousing: star schema with very wide fact table, but typically you focus on few columns at a time
- btree indexes for oltp, bitmaps + btree for warehouse

### Storage components of a database

- the heap file: this is where the rows or columns are stored
- regular relational databases use row oriented heap files
- an index file(s): this is where the index for a particular facet is stored
- sometimes you have a clustered index ( all data stored in index) or covering index (some data is stored in index)
- the WAL or write-ahead log: this is used to handle transactions


### Simple start

- start with index for key-value data
- aka dictionary
- in memory you are done. the index IS the database
- hash tables are no good for range queries

In [5]:
database=dict()
database['rahul']="aged"
database['pavlos']="ancient"
database['kobe']="stillyoung"
database['kobe']

'stillyoung'

### Doing it on disk

- in the dict in memory, store a file offset instead
- this "HEAP" file is an append only file, and thus also acts as a "LOG"
- if you update, simply append a new entry and change the offset in the dict
- this is an improvement to what you did for the linked-list based environment in the homework

![](https://dl.dropboxusercontent.com/u/75194/hashmaplog.png)

### Aside into memoryview and bytes and bytearrays and struct


#### Byteorder

![](http://ptgmedia.pearsoncmg.com/images/chap3_0131411551/elementLinks/03fig09.gif)

In [29]:
import sys
sys.byteorder

'little'

In [27]:
import struct
fmt = '<3s3sHH'#little endian, 2 seq 3 bytes, 2 unsigned shorts
with open("pcanim.gif", 'rb') as fd:
    readit = fd.read()
    msg = memoryview(readit) #no copy
header = msg[:10] # 10 byte view, no copy, imagine the savings
bytes(header)# finally a copy

b'GIF89a\xe8\x03\x90\x01'

In [28]:
struct.unpack(fmt, header)#type/version/width/height

(b'GIF', b'89a', 1000, 400)

In [30]:
readit[0]

71

In [31]:
readit[0]=5

TypeError: 'bytes' object does not support item assignment

*bytearrays* as opposed to bytes, are read-write, which leads to being able to pre-allocate a "buffer", get a memoryview on it, and use the slice syntax.

In [32]:
import os.path
with open("pcanim.gif", 'rb') as fd:
    data = bytearray(os.path.getsize("pcanim.gif"))
    fd.readinto(data)
mv = memoryview(data)
mv[0]=5

This gives us a way to do something we couldn't achieve by any other means - read from a file (or receive from a socket) directly into the middle of some existing buffer

```python
buf = bytearray(...) # pre-allocated to the needed size
mv = memoryview(buf)
numread = f.readinto(mv[some_offset:])
```

In [6]:
import os.path
import sys
class Database():
    
    def __init__(self, file):
        self.file = file
        self.byteorder=sys.byteorder
        if not os.path.exists(file):
            self.fd = open(file, "xb+", buffering=0) #see https://docs.python.org/3/library/functions.html#open
            self.index={}
        else:
            self.fd = open(file, "r+b", buffering=0)
            with open(file+".idx") as fdi:
                items = [l.strip().split(':') for l in fdi.readlines()]
                self.index = {k:int(v) for k,v in items}
        self.readptr = self.fd.tell()
        self.fd.seek(0,2)
        self.writeptr = self.fd.tell()
        
        
    def set(self, x, v):
        if not isinstance(x, str):
            raise ValueError("Key must be a string")
        bin_x = x.encode('utf-8')
        sz_x=len(bin_x).to_bytes(1, byteorder=self.byteorder)
        if not isinstance(v, str):
            raise ValueError("Value must be a string")
        bin_v = v.encode('utf-8')
        sz_v=len(bin_v).to_bytes(1, byteorder=self.byteorder)
        try:
            self.index[x]=self.writeptr
            self.fd.seek(self.writeptr)
            print("currently", self.fd.tell())
            self.fd.write(sz_x+sz_v+bin_x+bin_v)
        except:
            del self.index[x]
        else:
            self.writeptr=self.fd.tell()
            
    def get(self, x):
        try:
            offset = self.index[x]
        except:
            raise ValueError("{} is not in index".format(x))
        bin_x = x.encode('utf-8')
        print("offset is", offset)
        self.readptr=offset
        self.fd.seek(self.readptr)#seek to that point. we dont load anything in memory yet
        sz_k = int.from_bytes(self.fd.read(1), byteorder=self.byteorder)
        sz_v = int.from_bytes(self.fd.read(1), byteorder=self.byteorder)
        self.fd.seek(sz_k,1)
        readit=self.fd.read(sz_v).decode('utf-8')
        print("now", self.fd.tell())
        return readit
        
    def close(self):
        fdi=open(self.file+".idx","w")
        fdi.write("\n".join([k+":"+str(v) for k,v in self.index.items()]))
        fdi.close()
        self.fd.close()
        
    def __del__(self):
        self.close()

In [10]:
!rm /tmp/test.db

In [11]:
db = Database("/tmp/test.db")
print(db.index)

{}


In [12]:
db.set("rahul", "aged")
db.set("pavlos", "aged")
db.set("kobe", "stillyoung")

currently 0
currently 11
currently 23


In [14]:
!cat /tmp/test.db

rahulagedpavlosaged
kobestillyoung

In [15]:
print(db.index)
db.get("pavlos")

{'rahul': 0, 'pavlos': 11, 'kobe': 23}
offset is 11
now 23


'aged'

In [16]:
db.set("rahul","young")

currently 39


In [17]:
db.get("rahul")

offset is 39
now 51


'young'

In [18]:
db.close()

In [19]:
db=Database("/tmp/test.db")
print(db.get("rahul"))
print(db.get("pavlos"))
print(db.get("kobe"))

offset is 39
now 51
young
offset is 11
now 23
aged
offset is 23
now 39
stillyoung


In [20]:
db.set("pavlos", "ancient")
db.index

currently 51


{'kobe': 23, 'pavlos': 51, 'rahul': 39}

In [21]:
print(db.get("rahul"))
print(db.get("pavlos"))
print(db.get("kobe"))

offset is 39
now 51
young
offset is 51
now 66
ancient
offset is 23
now 39
stillyoung


In [22]:
db.close()

In [23]:
!cat /tmp/test.db.idx

rahul:39
pavlos:51
kobe:23

In [24]:
!cat /tmp/test.db

rahulagedpavlosaged
kobestillyoungrahulyoungpavlosancient

### Transaction Processing or Analytics?

- Also known as OLTP vs OLAP/Warehousing
- small query size vs aggregates over large ones
- random writes from user input vs ordered ETL/stream
- end user (amazon site) vs analyst (you)
- GB to TB vs TB to PB

![](https://dl.dropboxusercontent.com/u/75194/ETL.png)

### Row Oriented Storage

- heapfile or clustered index is a set of rows
- we'll see details of this storage soon
- index could be a tree with appropriate pointer to heapfile offset

![](https://www.simple-talk.com/iwritefor/articlefiles/1844-f4cc85b0-9ddb-44cc-93ef-a742fcc4f279.jpeg)

### Heap file and index: Column oriented storage

- store values from each column together in separate storage
- lends itself to compression with bitmap indexes and run-length encoding
- this involves choosing an appropriate sort order
- the index then can be the data (great for IN and AND queries): there is no pointers to "elsewhere"
- compressed indexes can fit into cache and are usable by iterators
- bitwise AND/OR can be done with vector processing
- several different sort orders can be redundantly stored
- writing is harder: updating a row touches many column files
- but you can write an in-memory front sorted store (row or column), and eventually merge onto the disk

![](https://www.simple-talk.com/iwritefor/articlefiles/1844-4e2482bb-aaff-4ebd-8900-1946560479af.jpeg)

### Data cubes

- Basically a histogram of counts in bins for multiple fields
- can give you fast marginals and conditionals in any combination of dimensions
- expensive to update so only used for warehousing
- such histograms are used by query optimizers as well

### WAL for commits

![](https://dl.dropboxusercontent.com/u/75194/wal1.png)
![](https://dl.dropboxusercontent.com/u/75194/wal2.png)
![](https://dl.dropboxusercontent.com/u/75194/wal3.png)


### Relational Grammar of Data

- provide simple verbs for simple things. These are functions corresponding to common data manipulation tasks
( https://cran.rstudio.com/web/packages/dplyr/vignettes/introduction.html )

See https://gist.github.com/TomAugspurger/6e052140eaa5fdb6e8c0/ which has a comparison of r/dplyr and pandas. I stole and modified this table from there:

``dplyr`` has a small set of nicely defined verbs. I've listed their closest pandas verbs.


<table>
  <tr>
    <th><b>VERB</b></th>
    <th><b>dplyr</b></th>
    <th><b>pandas</b></th>
    <th><b>SQL</b></th>
  </tr>
  <tr>
    <td>QUERY/SELECTION</td>
    <td>filter() (and slice())</td>
    <td>query() (and loc[], iloc[])</td>
    <td>SELECT WHERE</td>
  </tr>
  <tr>
    <td>SORT</td>
    <td>arrange()</td>
    <td>sort()</td>
    <td>ORDER BY</td>
  </tr>
  <tr>
    <td>SELECT-COLUMNS/PROJECTION</td>
    <td>select() (and rename())</td>
    <td>[](__getitem__) (and rename())</td>
    <td>SELECT COLUMN</td>
  </tr>
  <tr>
    <td>SELECT-DISTINCT</td>
    <td>distinct()</td>
    <td>unique(),drop_duplicates()</td>
    <td>SELECT DISTINCT COLUMN</td>
  </tr>
  <tr>
    <td>ASSIGN</td>
    <td>mutate() (and transmute())</td>
    <td>assign</td>
    <td>ALTER/UPDATE</td>
  </tr>
  <tr>
    <td>AGGREGATE</td>
    <td>summarise()</td>
    <td>describe(), mean(), max()</td>
    <td>None, AVG(),MAX()</td>
  </tr>
  <tr>
    <td>SAMPLE</td>
    <td>sample_n() and sample_frac()</td>
    <td>sample()</td>
    <td>implementation dep, use RAND()</td>
  </tr>
  <tr>
    <td>GROUP-AGG</td>
    <td>group_by/summarize</td>
    <td>groupby/agg, count, mean</td>
    <td>GROUP BY</td>
  </tr>
  <tr>
    <td>DELETE</td>
    <td>?</td>
    <td>drop/masking</td>
    <td>DELETE/WHERE</td>
  </tr>
</table>

We'll tackle joins in the sql tutorial, but until then, here is a rough understanding for them:

https://blog.codinghorror.com/a-visual-explanation-of-sql-joins/

We'll use sqlite here (and recommend Postgres for production purposes). Still sqlite is great for on-disk large databases which wont fit into memory. 

Its also built into Python, but to use the [command line tool](https://www.sqlite.org/cli.html), I recommend you install it: https://www.sqlite.org/download.html. I also recommend you download and install the sqlite browser: http://sqlitebrowser.org .

Python implements a standard database API over all databases. Its called [DBAPI2](http://cewing.github.io/training.codefellows/lectures/day21/intro_to_dbapi2.html). It works across many SQL databases.

There is an even higher level API available, called [SQLAlchemy](http://www.sqlalchemy.org). Thoroughly recommend it, either in its direct relational form, or ORM form. Many things in Pandas use it to interface with databases. 